In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from PIL import Image
import re
import os
import cv2
from skimage.feature import graycomatrix, graycoprops
import pandas as pd

In [9]:
import pandas as pd

def dataloader(path):
    # image_files = os.listdir(image_dir)
    # image_paths = [os.path.join(image_dir, filename) for filename in image_files]

    data = {}
    for filename in os.listdir(path):
        if re.search(r'dusty', filename):
            file_path = os.path.join(path, filename)
            data[file_path] = 1
        else:
            file_path = os.path.join(path, filename)    
            data[file_path] = 0

    return data

In [18]:
path = "D:\FYP\dataset"
dataset = dataloader(path)

# print(X[0])
# print(dataset)
Tr = []
for path,label in dataset.items():
    img = cv2.imread(path)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hue = img_hsv[:,:,0]

    glcm = graycomatrix(hue,distances=[1],angles=[0,np.pi/2])
    contrast = graycoprops(glcm, prop='contrast')
    homogeneity = graycoprops(glcm, prop='homogeneity')

    X_feature = np.array([contrast[0,0], contrast[0,1], homogeneity[0,0], homogeneity[0,1], label])
    Tr.append(X_feature)



In [76]:
Tr = np.array(Tr)

# np.savetxt('Features.csv', Tr, delimiter=',', fmt='%f')

df = pd.DataFrame(Tr)

excel_file_path = 'Features.xlsx'

# Save the DataFrame as an Excel file
df.to_excel(excel_file_path, index=False)

In [7]:
np.random.shuffle(Tr)
print(Tr)

[[ 46.039658  38.489357   0.919913   0.940825   1.      ]
 [106.815994 111.650081   0.863068   0.846262   1.      ]
 [ 26.247983  25.291152   0.851602   0.890788   0.      ]
 ...
 [110.942977  81.048909   0.891908   0.920918   1.      ]
 [ 25.850534  15.626874   0.834089   0.928595   1.      ]
 [ 39.161923  21.202847   0.765549   0.8562     0.      ]]


In [8]:
X = Tr[:,:4]
y = Tr[:,-1]
print(X.shape)

(783, 4)


In [9]:
def feature_vector(image):
    img_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hue = img_hsv[:,:,0]
    # cv2.imshow("window_name", hue)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    glcm = graycomatrix(hue,distances=[1],angles=[0,np.pi/2])
    contrast = graycoprops(glcm, prop='contrast')
    homogeneity = graycoprops(glcm, prop='homogeneity')

    X_feature = np.array([contrast[0,0], contrast[0,1], homogeneity[0,0], homogeneity[0,1]])

    return X_feature

In [23]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Build a neural network model
model = keras.Sequential([
    keras.layers.Dense(10, input_dim=4, activation='relu'),  # Input layer with 4 features
    keras.layers.Dense(20, activation='relu'),  # First hidden layer
    keras.layers.Dense(20, activation='relu'),  # Second hidden layer
    keras.layers.Dense(1, activation='sigmoid')  # Output layer with sigmoid activation for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=1)

# Make predictions
y_pred = (model.predict(X_test) > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Epoch 1/50
55/55 [==============================] - 1s 1ms/step - loss: 1.0382 - accuracy: 0.4544
Epoch 2/50
55/55 [==============================] - 0s 1ms/step - loss: 0.6532 - accuracy: 0.5858
Epoch 3/50
55/55 [==============================] - 0s 1ms/step - loss: 0.6329 - accuracy: 0.6077
Epoch 4/50
55/55 [==============================] - 0s 1ms/step - loss: 0.6310 - accuracy: 0.6369
Epoch 5/50
55/55 [==============================] - 0s 1ms/step - loss: 0.6328 - accuracy: 0.6131
Epoch 6/50
55/55 [==============================] - 0s 1ms/step - loss: 0.6377 - accuracy: 0.6442
Epoch 7/50
55/55 [==============================] - 0s 1ms/step - loss: 0.6270 - accuracy: 0.6277
Epoch 8/50
55/55 [==============================] - 0s 1ms/step - loss: 0.6234 - accuracy: 0.6606
Epoch 9/50
55/55 [==============================] - 0s 1ms/step - loss: 0.6113 - accuracy: 0.6588
Epoch 10/50
55/55 [==============================] - 0s 1ms/step - loss: 0.6217 - accuracy: 0.6332
Epoch 11/50
55/55 [

In [29]:
image = cv2.imread("D:\FYP\dataset\dusty361.jpg")

x = feature_vector(image)
x = x.reshape(1, -1)

predictions = model.predict(x)

print(predictions)


1/1 [==============================] - 0s 16ms/step
[[0.6267848]]
